In [93]:
# Libraries
import numpy as np
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
import re
import os
from urllib.request import urlopen
import time
from concurrent.futures import ThreadPoolExecutor

import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from collections import Counter
from functools import reduce
import json

# 1. Data collection

## 1.1. Get the list of master's degree courses
We created a file named 'urls.txt' that contains all the urls associated with the url of each master page.
for reaching such purpose, we iterate over all 400 pages and took the link for every 15 urls of each page.
we stored all urls in 'urls.txt' file.

In [2]:
f = open("urls.txt","w") # First we create a txt file where we can write the URLs  #  w means writing mode
for i in range(1, 401): #from first page to page 400
    url = f"https://www.findamasters.com/masters-degrees/msc-degrees/?PG={i}" #pages can be scrolled by changing the number after PG
    result = requests.get(url) # as we have done in class
    soup = BeautifulSoup(result.text, 'html.parser') # to get the html of each page

    for link in soup.find_all(class_ = re.compile('courseLink')): #as in class to get each tag of the page which belongs to class courseLink
        c = (link.get("href"))  # url of each page in the i-th page
        f.write("https://www.findamasters.com/"+c) #writing the rows
        f.write("\n")
f.close()
print('The "urls.txt" file is generated!')

The "urls.txt" file is generated!


### 1.2. Crawl master's degree pages

We wrote a function named 'download_url'.
Since the FindMaster website blocks us for 70 seconds for every (20 to 22) requests we send, we use 'time.sleep(70)' to wait and then resend the http get request. 
we also omit to download the http files that their directory are already existed.

for sending http get requests asynchronously, we can use async and await methods and take the advantage of using "aiohttp" library. the other way is to use ThreadPoolExecutor function executer. 
It means that we store the executer command in a variable named 'future_to_url' that we are able to call in the future.
The ThreadPoolExecutor is a built-in Python module that provides managing a pool of worker threads. It allows us to submit tasks to the pool, which are then executed by one of the worker threads in the pool.

In [2]:
from concurrent.futures import ThreadPoolExecutor

# Function to download and save HTML for a given URL
def download_url(url, folder_path, page_number):
    # Create a folder for each page if it doesn't exist
    page_folder = os.path.join(folder_path, f"page_{page_number}")
    if os.path.exists(page_folder):
        # uncomment the below code to see which pages are skiped, cause they have already been downloaded.
        # print(f"Skipping Page: {page_number} - Folder already exists.")
        return

    try:
        response = requests.get(url) # Send a GET request to the URL
        response.raise_for_status()  # Raise an exception for bad responses 

        # Create a folder for each page if it doesn't exist
        os.makedirs(page_folder, exist_ok=True)

        # Save the HTML content to a file
        file_path = os.path.join(page_folder, f"html_{page_number}.html")
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(response.text)
        print(f"Downloaded page {page_number}: {url}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download page {page_number}: {url}")
        print(f"Error: {e}")
        print("Retrying in 70 seconds...")
        time.sleep(70)  # Wait for 10 seconds before retrying
        download_url(url, folder_path, page_number)  # Retry the download

# Read all URLs one by one
with open('urls.txt', 'r') as urls_file:
    urls = urls_file.read().splitlines()

output_folder = 'HTML_folders' # Store all HTML files into this directory.

# We can use ThreadPoolExecutor for sending http requests asynchronously. 
# However, Since the FindMaster website blocks us for 70 seconds for every (20 to 22) requests we send, 
# the max_workers in below code assigned to number 1. So it sends requests synchronously.
with ThreadPoolExecutor(max_workers=1) as executor:
    # Enumerate through each URL and submit download tasks to the executor
    future_to_url = {executor.submit(download_url, url, output_folder, page_number): url for page_number, url in enumerate(urls, start=1)}

print("All HTML files are stored in the HTML_folders directory.")

All HTML files are stored in the HTML_folders directory.


### 1.3 Parse downloaded pages
Here we create a '.tsv' file including the following columns for each of the HTML files.

1. Course Name (to save as ```courseName```): string;
2. University (to save as ```universityName```): string;
3. Faculty (to save as ```facultyName```): string
4. Full or Part Time (to save as ```isItFullTime```): string;
5. Short Description (to save as ```description```): string;
6. Start Date (to save as ```startDate```): string;
7. Fees (to save as ```fees```): string;
8. Modality (to save as ```modality```):string;
9. Duration (to save as ```duration```):string;
10. City (to save as ```city```): string;
11. Country (to save as ```country```): string;
12. Presence or online modality (to save as ```administration```): string;
13. Link to the page (to save as ```url```): string.

Then, we merge all those files together to generate our final dataset.

In [5]:
current_path = os.getcwd()
# '/Users/armanfeili/Arman/Sapienza Courses/ADM/Homeworks/HW3/phase-2/ADM-HW3/HTML_folders'

for i in range(1,6001):
    # os.chdir(r'C:\Users\susan\Documents\DS\ADM\HW3\ADM-HW3\HTML_folders\page_'+str(i)) #change directories
    os.chdir(r'/Users/armanfeili/Arman/Sapienza Courses/ADM/Homeworks/HW3/phase-3/ADM-HW3/HTML_folders/page_'+str(i)) #change directories
    
    for filename in os.listdir(os.getcwd()): # get all the files in a folder
        if filename.endswith(".tsv"): continue # tsv file is already generated.
        elif filename.endswith(".html"): # if file extension is .html
            with open(os.path.join(os.getcwd(), filename), 'r',encoding='utf-8') as f: # open each file into a folder
                soup = BeautifulSoup(f,'html.parser') # get the html file by each file 
                out=[] # initialize a list where we append all the informations parsed from each html file

                # 1  Course Name
                courseName = soup.find_all(class_=re.compile("course-header__course-title"))
                out.append(courseName[0].text.strip() if courseName else "") #text.strip to eliminate strange simbols for the space
                # 2  University
                universityName = soup.find_all(class_=re.compile("course-header__institution"))
                out.append(universityName[0].text if universityName else "")
                # 3  Faculty
                facultyName = soup.find_all(class_=re.compile("course-header__department"))
                out.append(facultyName[0].text if facultyName else "")
                # 4  Full or Part Time
                isItFullTime = soup.find_all(class_=re.compile("concealLink"))
                out.append(isItFullTime[0].text if isItFullTime else "")
                # 5  Short Description
                description = soup.find_all(class_=re.compile("course-sections__content"))
                out.append(description[0].text.replace('\n', '') if description else "")
                # 6  Start Date
                startDate = soup.find_all(class_=re.compile("key-info__start-date"))
                out.append(startDate[0].text if startDate else "")
                # 7  Fees 
                fees_elements = soup.find_all(class_=re.compile("course-sections__fees")) # taking the fee
                fees_text = fees_elements[0].text.replace('\n', '') if fees_elements else "" 
                cleaned_fees = re.sub(r'Fees', '', fees_text)  # To not "Fees" at the beginning 
                out.append(cleaned_fees.strip() if cleaned_fees else "")
                # 8  Modality
                modality = soup.find_all(class_=re.compile("key-info__qualification"))
                out.append(modality[0].text if modality else "")
                # 9  Duration
                duration = soup.find_all(class_=re.compile("key-info__duration"))
                out.append(duration[0].text if duration else "")
                # 10  City
                city = soup.find_all(class_=re.compile("course-data__city"))
                out.append(city[0].text if city else "")
                # 11  Country
                country = soup.find_all(class_=re.compile("course-data__country"))
                out.append(country[0].text if country else "")
                # 12  Presence or online modality
                # We have seen that some courses has both online or oncampus modality, one of them is "Master of Business Administration"
                on_campus_elements = soup.find_all(class_=re.compile("course-data__on-campus"))
                online_elements = soup.find_all(class_=re.compile("course-data__online"))
                if on_campus_elements and online_elements:
                    out.append("both")
                else:
                    out.append(on_campus_elements[0].text if on_campus_elements else online_elements[0].text if online_elements else "Nan")
                # 13  Link to the page
                out.append(soup.find('link', {'rel': 'canonical'}).get('href') if soup.find('link', {'rel': 'canonical'}) else "Nan")
                f.close()
                
                # Creating file .tsv
                l = ['courseName','universityName','facultyName','isItFullTime','description','startDate','fees','modality','duration',
                    'city','country','administration','url']
                with open(filename+'.tsv','w',encoding='utf-8') as tsv:
                    tsv_output = csv.writer(tsv, delimiter='\t')
                    tsv_output.writerow(l)
                    tsv_output.writerow(out)
    os.chdir('..')  

print("All HTML files have been read and all .tsv files have been generated.")

All HTML files have been read and all .tsv files have been generated.


In [6]:
data=[]
# to merge all the .tsv files
for i in range(1,6001):
    # os.chdir(r'./HTML_folders/page_'+str(i)) #change directories
    os.chdir(r'/Users/armanfeili/Arman/Sapienza Courses/ADM/Homeworks/HW3/phase-3/ADM-HW3/HTML_folders/page_'+str(i)) #change directories
    for filename in os.listdir(os.getcwd()):
        if filename.endswith(".tsv"):
            a = pd.read_csv(filename,sep='\t')
            data.append(a)
    os.chdir('..')
data=pd.concat(data,ignore_index=True)   
data.to_csv('../dataset.tsv',sep='\t',index=False) # Saving the big one
print("dataset.tsv file has been generated as the main dataset.")

dataset.tsv file has been generated as the main dataset.


In [7]:
# An illustration to the dataset:
data.head(5)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
2,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,Full time,"Our Accounting, Accountability & Financial Man...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
3,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wit...,September,Please see the university website for further ...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
4,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Full time,Join us for an online session for prospective ...,September,Please see the university website for further ...,MSc,One year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...


## 2. Search Engine

In [53]:
# Load the dataset we are going to work with
data = pd.read_table(r"dataset.tsv")

### 2.0 Preprocessing

### 2.0.0)  Preprocessing the text

In [54]:
# This function takes a text, removes special cases, punctuations and stop-words then
# it applies stemming and finally returns the preprocessed words separated by commas.
# TODO maybe they want to apply this to the whole data, not just description
def preprocess_description(description_text):

    # Handle the cases of float type (there are 24)
    if type(description_text) != str:
        return ""

    # Remove all special chars and punctuations
    description_text = re.sub("[^a-z A-Z ]+","", description_text)

    # Convert everything in lowercase
    description_text = description_text.lower()

    # Remove stopwords using nltk package
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(description_text)
    words = [word for word in words if word not in stop_words]

    # Apply stemming using ntlk package
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Separate words with commas
    words = ','.join(words)

    return words

In [55]:
# Create and fill new column where the preprocessed descriptions wil be stored:
data["clean_description"] = data["description"].apply(preprocess_description)

In [56]:
# Show different fields
data[["description", "clean_description"]].head(10)

,description,clean_description
0,3D visualisation and animation play a role in ...,"visualis,anim,play,role,mani,area,popular,medi..."
1,Businesses and governments rely on sound finan...,"busi,govern,reli,sound,financi,knowledg,underp..."
2,"Our Accounting, Accountability & Financial Man...","account,account,financi,manag,msc,cours,provid..."
3,Embark on a professional accounting career wit...,"embark,profession,account,career,academ,ground..."
4,Join us for an online session for prospective ...,"join,us,onlin,session,prospect,student,find,ms..."
5,The Advanced Chemical Engineering MSc at Leeds...,"advanc,chemic,engin,msc,leed,build,core,founda..."
6,Programme overviewThe Advanced Master in Finan...,"programm,overviewth,advanc,master,financi,mark..."
7,Programme overviewThe Advanced Master in Innov...,"programm,overviewth,advanc,master,innov,strate..."
8,Progress your career as a physiotherapist with...,"progress,career,physiotherapist,within,nh,priv..."
9,Goal of the pro­grammeWould you like to be inv...,"goal,programmewould,like,involv,find,solut,fut..."


### 2.0.1) Preprocessing the fees column

In [82]:
# This function was provided by ChatGPT.
# It downloads the latest exchange rates (wrt United States Dollars) from openexchangerates.org.
# It returns a dictionary of the form {exchange_code : exchange_rate}
def get_latest_exchange_rates(app_id):

    base_url = "https://openexchangerates.org/api/latest.json"
    params = {"app_id": app_id}
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.json().get("rates")
    else:
        print(f"Failed to fetch exchange rates. Status code: {response.status_code}")
        return None


# Dictionary provided by ChatGPT, I changed some symbols that were not present.
# It will be used to convert a currency symbol into currency code.

currency_symbol_to_code = {
    '$': 'USD',  # United States Dollar
    '€': 'EUR',  # Euro
    "EURO": "EUR", # Euro
    '¥': 'JPY',  # Japanese Yen
    '£': 'GBP',  # British Pound Sterling
    'A$': 'AUD',  # Australian Dollar
    'C$': 'CAD',  # Canadian Dollar
    'CHF': 'CHF',  # Swiss Franc
    'KR': 'SEK',  # Swedish Krona
    'NZ$': 'NZD',  # New Zealand Dollar
    # Add more symbols and codes as needed
}

# This function handles the preprocessing of the "fees" field
def preprocess_fees(fees_text):

    # Handles the case of "nan"
    if type(fees_text) != str:
        return None

    # Preallocate all the fees found in text
    total_fees = []

    # Symbols we are looking for
    symbols = "GBP|USD|ISK|£|\$|₹|¥|₪|₽|₩|₦|₴|﷼|€|Euro"

    # Match the (symbol, number) case
    left_symbol_matches = re.findall(fr'(?:{symbols})+\s*[0-9]+[,.]?[0-9]*', fees_text, flags=re.IGNORECASE)

    # Match the (number,  symbol) case
    right_symbol_matches = re.findall(fr'[0-9]+[,.]?[0-9]*\s*(?:{symbols})+', fees_text, flags=re.IGNORECASE)

    # Merge them
    matches = left_symbol_matches + right_symbol_matches

    # If we got no matches returns None
    if len(matches) == 0 :
        return None

    for match in matches:
        # Remove "," or "." and change to the right type
        number = re.findall('([0-9]+)[.,]*([0-9]*)', match)[0]
        number = float(number[0] + number[1])

        # Isolate symbol and upper case it (to match exchange_rates_dict codes)
        symbol = re.findall(fr'(?i)({symbols})', match)[0].upper()

        # Transform symbol into code (if not already a code)
        if symbol in currency_symbol_to_code.keys():
            symbol = currency_symbol_to_code[symbol]

        # Change into USD using the exchange_rates dictionary and append to fees
        total_fees.append(number * exchange_rates_dict[symbol])

    # Take the max fee and return it
    max_fee = round(max(total_fees))
    return max_fee


In [83]:
# Load latest exchange rates
my_app_id = "1457fcd3d536441baad3ce7918b5025b"
exchange_rates_dict = get_latest_exchange_rates(my_app_id)

# Show exchange_rates_dict
count = 0
print("{")
for key, value in exchange_rates_dict.items():
    if count < 5:
        print(f'{key}: {value}')
        count += 1
    else:
        break
print("}")

{
AED: 3.673
AFN: 68.721095
ALL: 95.196661
AMD: 400.751217
ANG: 1.793489
}


In [84]:
# Save preprocessed fees into a new column
# #TODO with this preprocessing we have 80% of NaN's, maybe we could do better?
data["fees_USD"] = data["fees"].apply(preprocess_fees)

In [85]:
data[["fees", "fees_USD"]].head(10)

,fees,fees_USD
0,Please see the university website for further ...,NaN
1,"UK: £18,000 (Total) International: £34,750 (To...",27896.0
2,Please see the university website for further ...,NaN
3,Please see the university website for further ...,NaN
4,Please see the university website for further ...,NaN
5,"UK: £13,750 (Total) International: £31,000 (To...",24886.0
6,18.000 €,16488.0
7,18.000 €,16488.0
8,Please see the university website for further ...,NaN
9,Tuition fee per year (non-EU/EEA students): 15...,13740.0


## 2.1. Conjunctive query

### 2.1.1) Create your index!

In [86]:
# Store all terms contained in the "clean_description" as a set
terms_set = set(','.join(data["clean_description"]).split(","))

# Create a dict that associate each term to a unique id
terms_id_dict = {key: value for value, key in enumerate(terms_set)}

In [87]:
# Here we create the inverted index dictionary.

# Preallocate a dictionary with the form: {term_id : []}
inverted_index_dict = {i : [] for i in range(len(terms_id_dict))}

# Iterating over all terms and texts in the "clean_description" field
for i,text in enumerate(data["clean_description"]):
    text_list = text.split(",")
    for term in text_list:

        # Get term id
        term_id = terms_id_dict[term]

        # Add document id "i" to the term_id list
        inverted_index_dict[term_id].append(i)

In [88]:
# Show inverted_index_dict structure
print("{")
count = 0
for key, value in inverted_index_dict.items():
    if count < 5:
        print(f'{key}: {value}')
        count += 1
    else:
        break
print("}")

{
0: [487, 532, 694, 770, 796, 908, 950, 1067, 1421, 1779, 1786, 1922, 2098, 2936, 2938, 3221, 3676, 3735, 4362, 4400, 4488, 4597, 5440, 5742]
1: [3942]
2: [4145]
3: [4979]
4: [25, 52, 65, 91, 155, 247, 278, 285, 325, 387, 452, 512, 517, 650, 710, 741, 750, 762, 808, 1034, 1050, 1449, 1577, 1667, 1709, 1734, 1741, 1882, 1882, 1882, 1882, 1964, 2282, 2407, 2439, 2583, 2656, 2767, 2783, 2784, 2814, 2853, 2929, 2929, 2929, 2998, 3076, 3076, 3093, 3301, 3446, 3461, 3587, 3684, 3926, 3926, 3926, 4068, 4075, 4077, 4111, 4142, 4175, 4329, 4374, 4375, 4394, 4417, 4418, 4418, 4419, 4419, 4419, 4419, 4420, 4420, 4420, 4420, 4420, 4421, 4421, 4421, 4421, 4421, 4421, 4473, 4560, 4610, 4612, 4629, 4725, 4726, 4920, 4938, 4997, 5070, 5123, 5132, 5137, 5150, 5150, 5150, 5150, 5188, 5357, 5364, 5449, 5801, 5931, 5959]
}


### 2.1.2) Execute the query

In [89]:
# This function takes a query as a input and returns the most affine docs:

def naive_search_engine(query):
    # Apply same preprocessing done for descriptions and split wrt ","
    query = preprocess_description(query).split(",")

    # For each term in query get all the docs ids that contain it as a set
    query_docs = [set(inverted_index_dict[terms_id_dict[term]]) for term in query]

    # Select the docs ids that contain all the query term, and sort them
    query_docs = set.intersection(*query_docs)
    query_docs = list(sorted(query_docs))

    # Return selected columns of those docs
    result  = data.iloc[query_docs, [0,1,4,12]]

    return result

In [90]:
naive_search_engine("advanced knowledge")

,courseName,universityName,description,url
1,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound finan...,https://www.findamasters.com/masters-degrees/c...
4,Addictions MSc,King’s College London,Join us for an online session for prospective ...,https://www.findamasters.com/masters-degrees/c...
12,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stud...,https://www.findamasters.com/masters-degrees/c...
48,Civil Engineering MSc,University of Greenwich,Meet the future demands of the construction in...,https://www.findamasters.com/masters-degrees/c...
86,Economics - MSc,University of Leeds,Our MSc Economics allows you to apply economic...,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...
5909,Master of Science/Postgraduate Diploma in Envi...,The Hong Kong University of Science and Techno...,The program is meant to meet the needs of prac...,https://www.findamasters.com/masters-degrees/c...
5937,Master Sociology – Social and Economic Psychology,University of Cologne,This programme provides you with:a solid found...,https://www.findamasters.com/masters-degrees/c...
5957,Masters in Economics,University of Lisbon,OBJECTIVESThe MSc in Economics aims to provide...,https://www.findamasters.com/masters-degrees/c...
5963,Master's in Global and European Politics,European School of Political and Social Scienc...,Europe and the EU in a changing worldOur inter...,https://www.findamasters.com/masters-degrees/c...


## 2.2) Conjunctive query & Ranking score


For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

Find all the documents that contain all the words in the query.
Sort them by their similarity with the query.
Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.
To solve this task, you must use the tfIdf score and the Cosine similarity. The field to consider is still the description. Let's see how.

### 2.2.1) Inverted index

Your second Inverted Index must be of this format:

{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],
term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
...}

In [91]:
# Here we create the second inverted index dictionary.

# Preallocate a dictionary with the form: {term_id : []}
inverted_index_dict_with_scores = {i : [] for i in range(len(terms_id_dict))}

# Iterating over all terms and texts in the "clean_description" field
for i,text in enumerate(data["clean_description"]):
    text_list = text.split(",")

    # Set here has the purpose of selecting unique terms only.
    # We don't want to insert in inverted_2 dict multiple times the same score
    for term in set(text_list):

        # Get term id
        term_id = terms_id_dict[term]

        # Get idf score: total number of documents / number of documents term is in
        term_idf = len(data["clean_description"]) / len(inverted_index_dict[term_id])

        # Get tf score: number of times term appears in text / total number of terms in text
        term_tf =  sum([word == term for word in text_list]) / len(text_list)

        # Compute tfidf score
        term_tfidf = term_tf * term_idf

        # Update new_inverted list
        inverted_index_dict_with_scores[term_id].append((i , term_tfidf))

In [92]:
# Show inverted_index_dict_with_scores structure
print("{")
count = 0
for key, value in inverted_index_dict_with_scores.items():
    if count < 5:
        print(f'{key}: {value}')
        count += 1
    else:
        break
print("}")

{
0: [(487, 250.0), (532, 250.0), (694, 250.0), (770, 250.0), (796, 250.0), (908, 250.0), (950, 250.0), (1067, 250.0), (1421, 250.0), (1779, 250.0), (1786, 250.0), (1922, 250.0), (2098, 250.0), (2936, 250.0), (2938, 250.0), (3221, 250.0), (3676, 250.0), (3735, 250.0), (4362, 250.0), (4400, 250.0), (4488, 250.0), (4597, 250.0), (5440, 250.0), (5742, 250.0)]
1: [(3942, 103.44827586206897)]
2: [(4145, 81.08108108108108)]
3: [(4979, 37.5)]
4: [(25, 1.4742014742014744), (52, 1.048951048951049), (65, 0.8141112618724559), (91, 0.9244992295839753), (155, 0.9404388714733543), (247, 1.3303769401330379), (278, 0.9917355371900827), (285, 1.948051948051948), (325, 0.7177033492822966), (387, 0.8797653958944281), (452, 1.0695187165775402), (512, 0.7905138339920948), (517, 3.896103896103896), (650, 0.9090909090909091), (710, 0.8797653958944281), (741, 0.6128702757916241), (750, 0.9244992295839753), (762, 0.9569377990430622), (808, 0.9090909090909091), (1034, 0.8941877794336811), (1050, 0.9404388714733

In [61]:
# Here we define our second search engine

# This function computes from scratch cosine similarity
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Calculate the norms
    norm_vec_1 = np.linalg.norm(vec1)
    norm_vec_2 = np.linalg.norm(vec2)

    # Compute the cosine similarity
    cos_sim = np.dot(vec1, vec2) / (norm_vec_1 * norm_vec_2)

    return cos_sim


def  top_k_search_engine(query, k):

    # Apply same preprocessing done for descriptions and split wrt ","
    query = preprocess_description(query).split(",")

    # Get query terms ids
    query_ids = [terms_id_dict[term] for term in query]

    # Calculate for each term in query its tfidf score
    query_idf = np.array([len(data["clean_description"]) / len(inverted_index_dict[term_id]) for term_id in query_ids])
    query_tf =  np.array([sum([word == term for word in query]) / len(query) for term in query]) # change this
    query_tfidf = query_idf * query_tf

    # Get the indexes of docs that contain all terms in query using inverted_1
    docs_ids = [set(inverted_index_dict[term_id]) for term_id in query_ids]
    appropriate_docs_ids = list(set.intersection(*docs_ids))

    # docs_tfidf will contain, for each id in appropriate_docs_ids, its tfidf vectorial representation.
    docs_tfidf = {i : [] for i in appropriate_docs_ids}

    # For each term in the query we retrieve its inverted_2 list of tuples
    for term_id in query_ids:
        list_of_tuples = inverted_index_dict_with_scores[term_id]
        for tuple_doc in list_of_tuples:

            # When we encounter a tuple with an appropriate doc id  we add its tfidf score in docs_tfidf
            if tuple_doc[0] in appropriate_docs_ids:
                docs_tfidf[tuple_doc[0]].append(tuple_doc[1])

    # Transform into a list
    docs_tfidf = list(docs_tfidf.values())

    # Compute cosine similarities between query_tfidf and each doc_tfidf
    cos_sims = [cosine_similarity(query_tfidf, doc_tfidf) for doc_tfidf in docs_tfidf]

    # Select all appropriate_docs_ids from data and specified columns
    result  = data.iloc[appropriate_docs_ids, [0,1,4,12]]

    # Add the cosine similarity score and sort the dataframe
    result["cos_sim"] = cos_sims
    result = result.sort_values(by='cos_sim', ascending=False)

    # Get, if possible, just the top k
    if k < result.shape[0]:
        result = result[:k]

    return result

In [62]:
top_k_search_engine("advanced knowledge", k = 10)

,courseName,universityName,description,url,cos_sim
3993,Geo-information Science and Earth Observation MSc,University of Twente,INTERESTED IN A CAREER IN SPATIAL DATA SCIENCE...,https://www.findamasters.com/masters-degrees/c...,1.0
2041,Clinical Ophthalmic Practice MSc,University College London,Register your interest in graduate study at UC...,https://www.findamasters.com/masters-degrees/c...,1.0
5120,LSE-Peking University Double Degree MSc Enviro...,London School of Economics and Political Science,Ask LSEOrganised jointly by LSE and Peking Uni...,https://www.findamasters.com/masters-degrees/c...,1.0
2436,Countering Extremist Crime and Terrorism MSc,University College London,Register your interest in graduate study at UC...,https://www.findamasters.com/masters-degrees/c...,1.0
4897,International Money and Banking - MSc,University of Birmingham,"The correlation between macroeconomics, bankin...",https://www.findamasters.com/masters-degrees/c...,1.0
1496,Biomaterials and Tissue Engineering MSc,University College London,Register your interest in graduate study at UC...,https://www.findamasters.com/masters-degrees/c...,1.0
764,Advanced Design and Manufacturing - MSc,University of Northampton,Our Advanced Design and Manufacturing MSc addr...,https://www.findamasters.com/masters-degrees/c...,1.0
3981,Geographical Information Systems and Environme...,University of Brighton,Environmental issues are a global concern. Our...,https://www.findamasters.com/masters-degrees/c...,1.0
2818,Diagnostic Radiography MSc,University College Cork,Our MSc Diagnostic Radiography programme is an...,https://www.findamasters.com/masters-degrees/c...,1.0
5576,Master of Chemistry (Leuven),KU Leuven,Breakthroughs in chemistry can change the text...,https://www.findamasters.com/masters-degrees/c...,1.0


## 5. BONUS: More complex search engine

The following pice of code, read the dataset, and then accepts queries from user in 2 ways.
1. User can search dataset based on 'course_name, university_name, university_city, fee_min, fee_max, countries, started_only, online_only'. User can specify none, all or some of them to retrive the desired rows.
2. User can Write a sentence as query that searches through course_name, university_name, university_city, and based on the most similarity, the data will be returned. For the other options, user should explicitly specify them as argument to the search_courses() function.

In [ ]:
import pandas as pd
from datetime import datetime

# Load the dataset we are going to work with
data = pd.read_table(r"dataset.tsv")  # Read the dataset from a tab-separated file

from collections import defaultdict  # Import defaultdict from the collections module
import math  # Import the math module for mathematical operations

def tokenize(text):
    if isinstance(text, str):
        return text.lower().split()  # Convert text to lowercase and split into tokens
    return []  # Return an empty list for non-string values

# Function to calculate cosine similarity
def cosine_similarity(query_tokens, doc_tokens):
    query_vector = defaultdict(int)  # Create a dictionary to store query token frequencies
    doc_vector = defaultdict(int)  # Create a dictionary to store document token frequencies

    # Calculate term frequencies for query and document
    for token in query_tokens:
        query_vector[token] += 1  # Increment token frequency in the query vector
    for token in doc_tokens:
        doc_vector[token] += 1  # Increment token frequency in the document vector

    # Compute dot product
    dot_product = sum(query_vector[token] * doc_vector[token] for token in query_tokens if token in doc_tokens)

    # Calculate magnitudes
    query_magnitude = math.sqrt(sum(query_vector[token] ** 2 for token in query_tokens))
    doc_magnitude = math.sqrt(sum(doc_vector[token] ** 2 for token in doc_tokens))

    # Avoid division by zero
    if query_magnitude != 0 and doc_magnitude != 0:
        similarity = dot_product / (query_magnitude * doc_magnitude)  # Calculate cosine similarity
        return similarity
    else:
        return 0.0  # Return 0 if either magnitude is zero

# Function to retrieve documents based on similarity
def retrieve_similar_documents(query, inverted_indexes, search_results):
    query_tokens = tokenize(query)  # Tokenize the query

    # Aggregate similarity scores for each document
    similarity_scores = defaultdict(float)  # Create a dictionary for similarity scores
    for token in query_tokens:
        if token in inverted_indexes:
            postings = inverted_indexes[token]
            for doc_id, relevance in postings:
                doc_tokens = tokenize(search_results.loc[doc_id, 'courseName'] + " " + 
                                      search_results.loc[doc_id, 'universityName'] + " " +
                                      search_results.loc[doc_id, 'city'])
                similarity = cosine_similarity(query_tokens, doc_tokens)
                similarity_scores[doc_id] += similarity * relevance  # Weighted sum of similarities

    # Sort documents by aggregated similarity
    sorted_docs = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_docs

def parse_custom_date(date):
    try:
        return datetime.strptime(date, "%B")  # Add the correct date format for parsing
    except ValueError:
        return pd.NaT  # Return NaT (Not a Timestamp) for invalid dates

def get_current_month_name():
    current_month_number = datetime.now().month
    return month_names[current_month_number - 1]  # Adjust index to match the list

def filter_started_courses(data):
    current_month_name = get_current_month_name()
    month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    current_month_index = month_names.index(current_month_name)

    data = data[data['startDate'].notna()]  # Omit rows where 'startDate' is NaT
    data['start_months'] = data['startDate'].apply(lambda x: x.split(', '))  # Split multiple months into a list

    def has_started(start_months):
        for month in start_months:
            if month in month_names:
                month_index = month_names.index(month)
                if month_index <= current_month_index:
                    return True
        return False

    data = data.copy()
    data.loc[:, 'has_started'] = data['start_months'].apply(has_started)  # Using .loc to assign the column

    data = data[data['has_started']]

    data.drop(columns=['start_months', 'has_started'], inplace=True)  # Drop helper columns

    return data


def filter_online_modality(data):
    data_copy = data.copy()  # Create a copy of the DataFrame to avoid chained assignment warnings
    return data_copy[data_copy['administration'].str.contains('Online', case=False, na=False, regex=False)]

def search_courses(data, query=None, course_name=None, university_name=None, university_city=None, fee_min=None, fee_max=None, countries=None, started_only=False, online_only=False):
    if query:

        # Create a dictionary to hold inverted indexes
        inverted_indexes = {}
        for index, row in search_results.iterrows():
            # Tokenize relevant fields
            course_tokens = tokenize(row['courseName'])
            university_tokens = tokenize(row['universityName'])
            city_tokens = tokenize(row['city'])
        
            # Update inverted indexes for each token
            for token in course_tokens + university_tokens + city_tokens:
                if token not in inverted_indexes:
                    inverted_indexes[token] = []
                inverted_indexes[token].append((index, 1))  # Assuming relevance of 1 for simplicity

        similar_docs = retrieve_similar_documents(query, inverted_indexes, search_results)

        # Retrieve top N documents based on similarity
        top_n = 50  # Number of top documents to retrieve
        top_documents = [doc_id for doc_id, similarity in similar_docs[:top_n]]
        relevant_results = search_results.loc[top_documents]
        # data = data[data.index.isin(top_documents)]
        data = relevant_results

    # Handling special case for fees (preprocess_fees function needs to be defined)
    data['fees'] = data['fees'].apply(lambda x: 0 if x == "Please see the university website for further information on fees for this course." else x)
    data["fees_USD"] = data["fees"].apply(preprocess_fees)  # Add preprocess_fees function

    # Filtering based on optional queries
    if course_name:
        data = data[data['courseName'].str.contains(course_name, case=False, na=False, regex=False)]
    if university_name:
        data = data[data['universityName'].str.contains(university_name, case=False, na=False, regex=False)]
    if university_city:
        data = data[data['city'].str.contains(university_city, case=False, na=False, regex=False)]

    # Filtering based on fee range
    if fee_min is not None:
        data = data[data['fees_USD'].astype(float).fillna(0) >= fee_min]
    if fee_max is not None:
        data = data[data['fees_USD'].astype(float).fillna(0) <= fee_max]

    # Filtering based on countries
    if countries:
        data = data[data['country'].isin(countries)]

    # Filter for courses that have started
    if started_only:
        data = filter_started_courses(data)

    # Filter for online modality
    if online_only:
        data = filter_online_modality(data)

    return data

### Example usage
- Pass your optional queries to the search_courses function
- Replace None with the desired search terms

In [ ]:
# Example usage:
filtered_data = search_courses(data,
                                query="Computer science courses in Newcastle",
                                fee_min=5000,  # Replace with your minimum fee
                                fee_max=10000,  # Replace with your maximum fee
                                countries=["United Kingdom"],  # List of desired countries
                                started_only=True, # Filter for courses that have started in the current month
                                online_only=True)  # Filter for online modality

filtered_data  # Displaying the filtered dataset

In [ ]:
# Example usage:
filtered_data = search_courses(data,
                                query="Portland universities",
                                fee_min=None,  # Replace with your minimum fee
                                fee_max=None,  # Replace with your maximum fee
                                countries=[],  # List of desired countries
                                started_only=False, # Filter for courses that have started in the current month
                                online_only=False)  # Filter for online modality

filtered_data  # Displaying the filtered dataset

In [ ]:
# Example usage:
search_results = search_courses(data,
                                course_name=None,
                                university_name=None,
                                university_city=None,
                                fee_min=None,  # Replace with your minimum fee
                                fee_max=None,  # Replace with your maximum fee
                                countries=[],  # List of desired countries
                                started_only=True, # Filter for courses that have started in the current month
                                online_only=False)  # Filter for online modality

# search_results[['courseName', 'universityName', 'url']]
search_results